# Phase 2 - Convert transactions → Monthly customer spend

## Step 2.1 Loading the libraries

In [48]:
import pandas as pd
import numpy as np

## Step 2.2 Loading the Clean data

In [49]:
df = pd.read_csv("data/cleaned_sales.csv")

## Step 2.3 Group by Customer & Month

In [50]:
monthly_revenue = (
    df.groupby(
        ['Customer ID', 'Customer', 'order_month', 'Region', 'Segment', 'Industry'],
        as_index=False
    )['Sales']
    .sum()
)

In [51]:
monthly_revenue.head()
monthly_revenue.shape

(4461, 7)

## Step 2.4 Renaming Sales to mrr (monthly recurrring revenue)

In [52]:
monthly_revenue = monthly_revenue.rename(columns={'Sales': 'mrr'})

# Phase 3 - Creating full monthly timeline

## Step 3.1 Find min & max month

In [53]:
min_month = monthly_revenue['order_month'].min()
max_month = monthly_revenue['order_month'].max()

## Step 3.2 Create full month range

In [54]:
all_months = pd.date_range(min_month, max_month, freq='MS')

## Step 3.3 Creating customer-month grid

In [55]:
customers = monthly_revenue[['Customer ID', 'Customer', 'Region', 'Segment', 'Industry']].drop_duplicates()

customer_month_grid = customers.merge(
    pd.DataFrame({'order_month': all_months}),
    how='cross'
)

## Step 3.4 Forcing datetime on BOTH sides
### Pandas refuses to merge when join keys don't match. 

In [56]:
customer_month_grid['order_month'] = pd.to_datetime(customer_month_grid['order_month'])
monthly_revenue['order_month'] = pd.to_datetime(monthly_revenue['order_month'])

In [57]:
## Step 3.5 Merge with revenue & filling missing MRR

In [58]:
full_mrr = customer_month_grid.merge( 
    monthly_revenue, 
    on=['Customer ID', 'Customer', 'Region', 'Segment', 'Industry', 'order_month'], 
    how='left' ) 
full_mrr['mrr'] = full_mrr['mrr'].fillna(0)

In [59]:
full_mrr.shape
monthly_revenue.shape

(4461, 7)

In [60]:
full_mrr[full_mrr['mrr'] == 0].sample(10)

,Customer ID,Customer,Region,Segment,Industry,order_month,mrr
12788,1035,Fannie Mae,APJ,Strategic,Finance,2021-09-01,0.0
24850,1072,Oracle,APJ,Strategic,Tech,2022-11-01,0.0
13044,1035,Fannie Mae,EMEA,SMB,Finance,2023-01-01,0.0
15929,1046,Volkswagen,AMER,SMB,Manufacturing,2023-06-01,0.0
24359,1071,News Corp.,APJ,SMB,Misc,2021-12-01,0.0
26117,1075,Philip Morris International,AMER,Strategic,Consumer Products,2020-06-01,0.0
20777,1059,Tesco,EMEA,SMB,Retail,2023-06-01,0.0
18465,1053,Alphabet,EMEA,SMB,Tech,2022-10-01,0.0
31502,1090,United Parcel Service,EMEA,Enterprise,Transportation,2021-03-01,0.0
27401,1079,Safeway,AMER,SMB,Retail,2023-06-01,0.0


# Phase 4 - Exporting clean MRR table 

In [63]:
full_mrr.to_csv("data/monthly_mrr.csv", index=False)

### this is perfect for SQL analysis